# Estimating Non-Mandatory Tour Scheduling

This notebook illustrates how to re-estimate the non-mandatory tour scheduling component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import larch as lx
import pandas as pd

lx.versions()

JAX not found. Some functionality will be unavailable.


{'larch': '6.0.32',
 'sharrow': '2.13.0',
 'numpy': '1.26.4',
 'pandas': '1.5.3',
 'xarray': '2024.3.0',
 'numba': '0.60.0'}

For this demo, we will assume that you have already run ActivitySim in estimation
mode, and saved the required estimation data bundles (EDB's) to disk.  See
the [first notebook](./01_estimation_mode.ipynb) for details.  The following module
will run a script to set everything up if the example data is not already available.

In [2]:
from est_mode_setup import prepare

prepare()

EDB directory already populated.


PosixPath('test-estimation-data/activitysim-prototype-mtc-extended')

# Load data and prep model for estimation

In [3]:
modelname = "non_mandatory_tour_scheduling"

from activitysim.estimation.larch import component_model

model, data = component_model(
    modelname,
    edb_directory=f"output-est-mode/estimation_data_bundle/{modelname}/",
    return_data=True,
)

loading from output-est-mode/estimation_data_bundle/non_mandatory_tour_scheduling/tour_scheduling_nonmandatory_coefficients.csv
loading from output-est-mode/estimation_data_bundle/non_mandatory_tour_scheduling/non_mandatory_tour_scheduling_SPEC.csv
loading from output-est-mode/estimation_data_bundle/non_mandatory_tour_scheduling/non_mandatory_tour_scheduling_alternatives_combined.parquet
loading from output-est-mode/estimation_data_bundle/non_mandatory_tour_scheduling/non_mandatory_tour_scheduling_choosers_combined.parquet


# Review data loaded from the EDB

The next (optional) step is to review the EDB, including the coefficients, utilities specification, and chooser and alternative data.

## Coefficients

In [4]:
data.coefficients

,value,constrain
coefficient_name,,
coef_dummy,1.000000,T
coef_subsequent_tour_must_start_after_previous_tour_for_this_purpose_ends,-999.000000,T
coef_free_flow_round_trip_auto_time_shift_effects_duration,0.004741,F
coef_shopping_tour_departure_shift_effects,-0.060150,F
coef_shopping_tour_duration_shift_effects,-0.120800,F
...,...,...
coef_escort_tour_duration_constants_4_to_5_hours,-2.880294,F
coef_escort_tour_duration_constants_6_to_7_hours,-2.973534,F
coef_escort_tour_duration_constants_8_to_10_hours,-3.020214,F


## Utility specification

In [5]:
data.spec

,Label,Description,Expression,Coefficient
5,util_subsequent_tour_must_start_after_previous...,Subsequent tour must start after previous tour...,(start < end_previous) & (tour_type_num > 1),coef_subsequent_tour_must_start_after_previous...
6,util_free_flow_round_trip_auto_time_shift_effe...,Free-flow round trip auto time shift effects -...,roundtrip_auto_time_to_work * duration,coef_free_flow_round_trip_auto_time_shift_effe...
7,util_shopping_tour_departure_shift_effects,Shopping tour - departure shift effects,@(_is_shopping) * df.start,coef_shopping_tour_departure_shift_effects
8,util_shopping_tour_duration_shift_effects,Shopping tour - duration shift effects,@(_is_shopping) * df.duration,coef_shopping_tour_duration_shift_effects
9,util_maintenance_tour_departure_shift_effects,Maintenance tour - departure shift effects,@(_is_othmaint) * df.start,coef_maintenance_tour_departure_shift_effects
...,...,...,...,...
91,util_escort_tour_duration_constants_4_to_5_hours,Escort Tour Duration Constants -- 4 to 5 hours,@(_is_escort) * (df.duration > 3) * (df.durati...,coef_escort_tour_duration_constants_4_to_5_hours
92,util_escort_tour_duration_constants_6_to_7_hours,Escort Tour Duration Constants -- 6 to 7 hours,@(_is_escort) * (df.duration > 5) * (df.durati...,coef_escort_tour_duration_constants_6_to_7_hours
93,util_escort_tour_duration_constants_8_to_10_hours,Escort Tour Duration Constants -- 8 to 10 hours,@(_is_escort) * (df.duration > 7) * (df.durati...,coef_escort_tour_duration_constants_8_to_10_hours
94,util_escort_tour_duration_constants_11_to_13_h...,Escort Tour Duration Constants -- 11 to 13 hours,@(_is_escort) * (df.duration > 10) * (df.durat...,coef_escort_tour_duration_constants_11_to_13_h...


## Chooser data

In [6]:
data.chooser_data

,tour_id,model_choice,override_choice,person_id,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,...,num_person_joint_tours,ptype,num_children,roundtrip_auto_time_to_work,num_mand,num_escort_tours,num_non_escort_tours,adult,start_previous,end_previous
0,1870,73,137,45,othdiscr,1,1,1,1,non_mandatory,...,0,4,0,0.0,0,0,1,True,5,5
1,20468,99,124,499,escort,1,1,1,1,non_mandatory,...,0,4,0,0.0,0,1,0,True,5,5
2,27055,146,169,659,social,1,1,1,1,non_mandatory,...,0,4,0,0.0,0,0,1,True,5,5
3,38877,37,125,948,escort,1,1,1,2,non_mandatory,...,0,4,0,0.0,0,1,1,True,5,5
4,38904,95,172,948,social,1,1,2,2,non_mandatory,...,0,4,0,0.0,0,1,1,True,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29664,310197956,125,127,7565803,shopping,1,1,1,1,non_mandatory,...,0,5,0,0.0,0,0,1,True,5,5
29665,310202376,126,5,7565911,othdiscr,1,1,2,2,non_mandatory,...,0,5,0,0.0,0,0,2,True,15,19
29666,310202384,149,129,7565911,shopping,1,1,1,2,non_mandatory,...,0,5,0,0.0,0,0,2,True,5,5
29667,310212634,86,146,7566161,shopping,1,1,1,1,non_mandatory,...,0,5,0,0.0,0,0,1,True,5,5


## Alternatives data

In [7]:
data.alt_values

,tour_id,start,end,duration,tdd,mode_choice_logsum,util_subsequent_tour_must_start_after_previous_tour_for_this_purpose_ends,util_free_flow_round_trip_auto_time_shift_effects_duration,util_shopping_tour_departure_shift_effects,util_shopping_tour_duration_shift_effects,...,util_escort_tour_arrival_constants_pm_peak_4,util_escort_tour_arrival_constants_evening,util_escort_tour_arrival_constants_late,util_escort_tour_duration_constants_0_to_1_hours,util_escort_tour_duration_constants_2_to_3_hours,util_escort_tour_duration_constants_4_to_5_hours,util_escort_tour_duration_constants_6_to_7_hours,util_escort_tour_duration_constants_8_to_10_hours,util_escort_tour_duration_constants_11_to_13_hours,util_escort_tour_duration_constants_14_to_18_hours
0,1870,5,5,0,0,0,False,0.0,0,0,...,False,False,False,False,False,False,False,False,False,False
1,1870,5,6,1,1,0,False,0.0,0,0,...,False,False,False,False,False,False,False,False,False,False
2,1870,5,7,2,2,0,False,0.0,0,0,...,False,False,False,False,False,False,False,False,False,False
3,1870,5,8,3,3,0,False,0.0,0,0,...,False,False,False,False,False,False,False,False,False,False
4,1870,5,9,4,4,0,False,0.0,0,0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3632342,310220296,21,22,1,185,0,False,0.0,0,0,...,False,False,False,False,False,False,False,False,False,False
3632343,310220296,21,23,2,186,0,False,0.0,0,0,...,False,False,False,False,False,False,False,False,False,False
3632344,310220296,22,22,0,187,0,False,0.0,0,0,...,False,False,False,False,False,False,False,False,False,False
3632345,310220296,22,23,1,188,0,False,0.0,0,0,...,False,False,False,False,False,False,False,False,False,False


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [8]:
model.doctor(repair_ch_av="-")

problem: chosen_but_not_available has (149 issues)


(<larch.Model (MNL) "None">,
 ┣ chosen_but_not_available:     altid   n        example rows
 ┃                           0       2   1               26680
 ┃                           1       7   1               15549
 ┃                           2       8   1                2727
 ┃                           3      12   1               18739
 ┃                           4      20   5  9872, 15764, 22362
 ┃                           ..    ...  ..                 ...
 ┃                           144   183  12  4208, 14096, 15246
 ┃                           145   184   5   1829, 8653, 13517
 ┃                           146   185  10     626, 5650, 9208
 ┃                           147   186   9    4563, 8308, 8746
 ┃                           148   187   7  8216, 13933, 19491
 ┃                           
 ┃                           [149 rows x 3 columns])

In [9]:
model.estimate(maxiter=900)

,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
coef_adjacent_window_exists_after_this_arrival_hour_first_tour_interaction,0.126593,0.126593,-0.025700,-25.0,25.0,0.0,0
coef_adjacent_window_exists_after_this_arrival_hour_second_plus_tour_interaction,-0.610258,-0.610258,-0.027340,-25.0,25.0,0.0,0
coef_adjacent_window_exists_before_this_departure_hour_first_tour_interaction,0.113506,0.113506,0.008442,-25.0,25.0,0.0,0
coef_adjacent_window_exists_before_this_departure_hour_second_plus_tour_interaction,0.386575,0.386575,-0.059300,-25.0,25.0,0.0,0
coef_adult_with_children_in_hh_arrive_19_21,0.281164,0.281164,0.336000,-25.0,25.0,0.0,0
...,...,...,...,...,...,...,...
coef_subsequent_of_2_plus_tours_for_same_purpose_duration_shift_effect,-0.281272,-0.281272,-0.173100,-25.0,25.0,0.0,0
coef_subsequent_tour_must_start_after_previous_tour_for_this_purpose_ends,-999.000000,-999.000000,-999.000000,-999.0,-999.0,0.0,1
coef_university_student_arrive_after_22,0.609651,0.609651,0.546600,-25.0,25.0,0.0,0


┣                x: coef_adjacent_window_exists_after_this_arrival_hour_first_tour_interaction               0.126593
┃                   coef_adjacent_window_exists_after_this_arrival_hour_second_plus_tour_interaction        -0.610258
┃                   coef_adjacent_window_exists_before_this_departure_hour_first_tour_interaction            0.113506
┃                   coef_adjacent_window_exists_before_this_departure_hour_second_plus_tour_interaction      0.386575
┃                   coef_adult_with_children_in_hh_arrive_19_21                                              0.281164
┃                                                                                                             ...    
┃                   coef_subsequent_of_2_plus_tours_for_same_purpose_duration_shift_effect                  -0.281272
┃                   coef_subsequent_tour_must_start_after_previous_tour_for_this_purpose_ends             -999.000000
┃                   coef_university_student_arrive_after_22                                                  0.609651
┃                   coef_visit_tour_departure_shift_effects                                                  0.080406
┃                   coef_visit_tour_duration_shift_effects                                                   0.140532
┃                   Length: 89, dtype: float64
┣          logloss: 3.8088398507438663
┣        d_logloss: coef_adjacent_window_exists_after_this_arrival_hour_first_tour_interaction             0.000034
┃                   coef_adjacent_window_exists_after_this_arrival_hour_second_plus_tour_interaction       0.000030
┃                   coef_adjacent_window_exists_before_this_departure_hour_first_tour_interaction         -0.000069
┃                   coef_adjacent_window_exists_before_this_departure_hour_second_plus_tour_interaction    0.000023
┃                   coef_adult_with_children_in_hh_arrive_19_21                                            0.000067
┃                                                                                                            ...   
┃                   coef_subsequent_of_2_plus_tours_for_same_purpose_duration_shift_effect                 0.000071
┃                   coef_subsequent_tour_must_start_after_previous_tour_for_this_purpose_ends              0.000000
┃                   coef_university_student_arrive_after_22                                               -0.000008
┃                   coef_visit_tour_departure_shift_effects                                               -0.000086
┃                   coef_visit_tour_duration_shift_effects                                                -0.000106
┃                   Length: 89, dtype: float64
┣              nit: 121
┣             nfev: 130
┣             njev: 121
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(seconds=95, microseconds=575519)
┣           method: 'slsqp'
┣          n_cases: 29669
┣ iteration_number: 121
┣          loglike: -103688.04725680027

### Estimated coefficients

In [10]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value,Constrained
Parameter,,,,,,
coef_adjacent_window_exists_after_this_arrival_hour_first_tour_interaction,0.127,0.0994,1.27,,0.00,
coef_adjacent_window_exists_after_this_arrival_hour_second_plus_tour_interaction,-0.610,0.0968,-6.30,***,0.00,
coef_adjacent_window_exists_before_this_departure_hour_first_tour_interaction,0.114,0.0850,1.34,,0.00,
coef_adjacent_window_exists_before_this_departure_hour_second_plus_tour_interaction,0.387,0.0868,4.45,***,0.00,
coef_adult_with_children_in_hh_arrive_19_21,0.281,0.0376,7.47,***,0.00,
coef_arrival_constants_am_peak,-1.62,0.104,-15.66,***,0.00,
coef_arrival_constants_early,0.00617,0.146,0.04,,0.00,
coef_arrival_constants_evening,0.00,0.00,NA,,0.00,fixed value
coef_arrival_constants_late,-0.817,0.0418,-19.58,***,0.00,


# Output Estimation Results

In [11]:
from activitysim.estimation.larch import update_coefficients
result_dir = data.edb_directory/"estimated"
update_coefficients(
    model, data, result_dir,
    output_file=f"{modelname}_coefficients_revised.csv",
);

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [12]:
model.to_xlsx(
    result_dir/f"{modelname}_model_estimation.xlsx", 
    data_statistics=False,
)

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [13]:
pd.read_csv(result_dir/f"{modelname}_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_dummy,1.000000,T
1,coef_subsequent_tour_must_start_after_previous...,-999.000000,T
2,coef_free_flow_round_trip_auto_time_shift_effe...,0.004326,F
3,coef_shopping_tour_departure_shift_effects,-0.061028,F
4,coef_shopping_tour_duration_shift_effects,-0.113223,F
...,...,...,...
85,coef_escort_tour_duration_constants_4_to_5_hours,-2.941739,F
86,coef_escort_tour_duration_constants_6_to_7_hours,-3.103903,F
87,coef_escort_tour_duration_constants_8_to_10_hours,-3.097454,F
88,coef_escort_tour_duration_constants_11_to_13_h...,-2.821199,F
